In [ ]:
# Chains in LangChain
# + LLMChain
# + Sequential Chains
#     - SimpleSequentialChain
#     - SequentialChain
# + Router Chain

In [1]:
def get_api_key():
    import os
    from dotenv import load_dotenv
    # Unset OPENAI_API_KEY if it exists
    os.environ.pop('OPENAI_API_KEY', None)
    # Provide path and ignore currently set environment variable
    dotenv_path = '.env'
    load_dotenv(dotenv_path=dotenv_path, override=True)
    # Access the environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    if api_key is None:
        print("API_KEY not found.")
    return api_key

In [2]:
import openai
openai.api_key = get_api_key()

In [6]:
!pip install pyarrow
import pandas as pd
df = pd.read_csv('Data.csv')


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

In [9]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.9, model=llm_model,openai_api_key=openai.api_key)

In [10]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

In [12]:
product = "Queen Size Sheet Set"
chain.invoke(product)

{'product': 'Queen Size Sheet Set', 'text': 'Queen Comforts'}

In [13]:
# Sequential Chains
'''
Sequential chain is another type of chains.

The idea is to combine multiple chains where the output of one chain is the input of the next chain

There are two type of sequential chains
1. SimpleSequentialChain: Single input/output
2. SequentialChain: multiple inputs/outputs
'''

'\nSequential chain is another type of chains.\n\nThe idea is to combine multiple chains where the output of one chain is the input of the next chain\n\nThere are two type of sequential chains\n1. SimpleSequentialChain: Single input/output\n2. SequentialChain: multiple inputs/outputs\n'

In [14]:
# https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html

from langchain.chains import sequential

In [15]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)
'''
Ah, in the context of Langchain, the | operator is used for a concept called "method chaining" or "pipeline". 
Langchain is a library that facilitates the construction and execution of language model pipelines. In Langchain, 
the | operator is used to connect different components or operations in a sequence, where the output of one 
component becomes the input to the next.

For example, in a Langchain pipeline like chain = prompt | llm, this means:

    prompt is an operation or a component that generates or processes some data.
    The | operator then passes the output of prompt to the next component in the chain.
    llm (which might stand for a language model like a large language model) takes the output from prompt and 
    performs its operation, perhaps generating text based on the prompt.

In this way, Langchain allows you to build complex language processing flows by chaining together simpler components,
 with each component performing a specific task and passing its output to the next. This design pattern is very powerful
for creating readable and maintainable code in data processing and manipulation tasks.
'''

# chain_one = first_prompt | llm

'\nAh, in the context of Langchain, the | operator is used for a concept called "method chaining" or "pipeline". \nLangchain is a library that facilitates the construction and execution of language model pipelines. In Langchain, \nthe | operator is used to connect different components or operations in a sequence, where the output of one \ncomponent becomes the input to the next.\n\nFor example, in a Langchain pipeline like chain = prompt | llm, this means:\n\n    prompt is an operation or a component that generates or processes some data.\n    The | operator then passes the output of prompt to the next component in the chain.\n    llm (which might stand for a language model like a large language model) takes the output from prompt and \n    performs its operation, perhaps generating text based on the prompt.\n\nIn this way, Langchain allows you to build complex language processing flows by chaining together simpler components,\n with each component performing a specific task and passin

In [16]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)
# chain_two = second_prompt | llm
overall_simple_chain = sequential.SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)

In [18]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
"Royal Rest Linens"
Royal Rest Linens offers high-quality and luxurious bedding products that ensure a comfortable and restful sleep experience.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Royal Rest Linens offers high-quality and luxurious bedding products that ensure a comfortable and restful sleep experience.'}

In [19]:
from langchain.chains import SequentialChain

review = df.Review[5]

print(review)

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?


In [20]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                    )

In [21]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [22]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [23]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [24]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [25]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in the store and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer expresses disappointment with the mediocre taste and poor foam retention of the product, suspecting whether it might be an old batch or counterfeit in comparison to the better-tasting ones they usually buy in-store.',
 'followup_message': "Réponse : Cher(e) critique, \n\nNous vous remercions d'avoir pris le temps de partager votre avis sur notre produit. Nous sommes vraiment désolés d'apprendre que vous avez été déçu(e) par son goût moyen et la faible tenue de sa mousse. Nous tenons à vous assurer que notre produit est authentique et que nous prenons ces commentaires très au sérieux.\n\nNotre équipe de productio

In [26]:
# Router Chain

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [27]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [29]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [30]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

print(destinations)
print(destinations_str)

['physics: Good for answering questions about physics', 'math: Good for answering math questions', 'History: Good for answering history questions', 'computer science: Good for answering computer science questions']
physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [31]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [32]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string name of the prompt to use or "DEFAULT"
    "next_inputs": string a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string name of the prompt to use or "DEFAULT"
    "next_inputs": string a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>


In [33]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)